# **Lection #7**:
### **Theme**: Патерны 2, Структурные 😭😲🤯
##### **Date**: 28.10.24

### **Паттерн** `Адаптер` 
- Преобразует интерфейс одного класса в интерфейс *другого*, который  
ожидают клиенты. Адаптер обеспечивает **совместную работу классов** с  
несовместимыми интерфейсами, которая без него была бы  
невозможна.  


In [2]:
class Target:
    def request(self) -> str:
        return "Target: the default target's behaviour"


class Adaptee:
    """
        я его адаптирую 🤯
    """
    def specific_request(self):
        return "Adaptee: special behavior"[::-1]


class Adapter(Target, Adaptee):
    def request(self) -> str:
        return f"Adapter: {self.specific_request()[::-1]}"


def client_code(target) -> None:
    print(target.request())


trgt = Target()
client_code(trgt)

# adptee = Adaptee()
adapter = Adapter()
client_code(adapter)

Target: the default target's behaviour
Adapter: Adaptee: special behavior


#### `Адаптер` через **композицию**


In [10]:
class AdapterComp(Target):
    def __init__(self, adaptee: Adaptee) -> None:
        self.adaptee = adaptee

    def request(self):
        return f"Adapter: {self.adaptee.specific_request()[::-1]}"

adaptee = Adaptee()
adapter2 = AdapterComp(adaptee)
client_code(adapter2)


Adapter: Adaptee: special behavior


### **Паттерн** `Фасад` 
- паттерн, *структурирующий* объекты.  
- **Фасад** определяет интерфейс более высокого уровня, который упрощает  
использование подсистемы.  


In [15]:
from __future__ import annotations

class Facade:
    def __init__(self, subsys1: Subsys1, subsys2: Subsys2) -> None:
        self._subsys1 = subsys1 or Subsys1()
        self._subsys2 = subsys2 or Subsys2()

    def operation(self) -> str:
        res = []
        res.append("Facade init subsys:")
        res.append(self._subsys1.operation1())
        res.append(self._subsys2.operation1())
        res.append("Facade orders subsys to perform the action:")
        res.append(self._subsys1.operation_n())
        res.append(self._subsys2.operation_z())
        return "\n".join(res)

class Subsys1:
    def operation1(str) -> str:
        return "Subsys 1 is ZZZZ!"

    def operation_n(str) -> str:
        return "Subsys 1 is CB0!"

class Subsys2:
    def operation1(str) -> str:
        return "Subsys 2 is VVVV!"

    def operation_z(str) -> str:
        return "Subsys 2 is ZOV!"

def client_code(facade: Facade):
    print(facade.operation())
    facade._subsys1.operation1()


sub1 = Subsys1()
sub2 = Subsys2()
facade = Facade(sub1, sub2)
client_code(facade)

Facade init subsys:
Subsys 1 is ZZZZ!
Subsys 2 is VVVV!
Facade orders subsys to perform the action:
Subsys 1 is CB0!
Subsys 2 is ZOV!


### **Паттерн** `Мост` 
- паттерн, *структурирующий* объекты.  
- отделить абстракцию от ее реализации так, чтобы то и другое  
можно было изменять независимо


In [18]:
from abc import ABC, abstractmethod

class Abstraction:
    def __init__(self, implementation: Implementation):
        self.implementation = implementation

    def operation(self):
        return f"Abstaction: Base operation with:\n{self.implementation.operation_imp()}"

class RefinedAbstraction(Abstraction):
    def operation(self):
        return f"Abstaction: Changed operation with:\n{self.implementation.operation_imp()}"


class Implementation(ABC):
    @abstractmethod
    def operation_imp(self):
        pass

class ConcreteImplementationA(Implementation):
    def operation_imp(self) -> str:
        return "ConcreteImplementation: result A"

class ConcreteImplementationB(Implementation):
    def operation_imp(self) -> str:
        return "ConcreteImplementation: result B"

def client_code(abstraction: Abstraction) -> None:
    print(abstraction.operation())

imp = ConcreteImplementationA()
abstraction = Abstraction(imp)
client_code(abstraction)

imp2 = ConcreteImplementationB()
abstraction2 = RefinedAbstraction(imp2)
client_code(abstraction2)


Abstaction: Base operation with:
ConcreteImplementation: result A
Abstaction: Changed operation with:
ConcreteImplementation: result B


### **Паттерн** `Компоновщик` 
- паттерн, *структурирующий* объекты.  
- компонует объекты в древовидные структуры для представления  
иерархий часть-целое. Позволяет клиентам единообразно  
трактовать индивидуальные и составные объекты.  


In [ ]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import List

class Component(ABC):
    @property
    def parent(self) -> Component:
        return self._parent


    @parent.setter
    def parent(self, parent: Component):
        self._parent = parent

    def add(self, component: Component):
        pass

    def remove(self, component: Component):
        pass

    def is_composite(self):
        return False

    @abstractmethod
    def operation(self) -> str:
        pass

class Leaf(Component):
    def operation(self) -> str:
        return "Leaf"

class Comosite(Component):
    def __init__(self) -> None:
        self._children: List[Component] = []

    def add(self, component: Component):
        pass

    def remove(self, component: Component):
        pass

    def is_composite(self):
        return False

Abstaction: Base operation with:
ConcreteImplementation: result A
Abstaction: Changed operation with:
ConcreteImplementation: result B
